<h1 style="color:#00A6D6;">Week 7: Device-independence</h1>

In class, we examined the device independence of QKD! Here we will have a look at something that is to some extent simpler - but essential for QKD: the generation of randomness. Here we have a weaker demand, we "just" want to produce a string $K$ such that

$$
\rho_{KE} \approx_{\epsilon} \frac{\mathbb{I}_K}{|K|} \otimes \rho_E\ ,
$$
but we do not care that Bob knows the same key $K$.

Of course, Bob being a good friend of Alice, will help her to ensure that $K$ is truly random even if her devices are untrusted. That is, we will assume that Alice and Bob again have to separated devices on which they can perform a Bell test such as CHSH. 

You may wonder whether this is at all useful. After all, Alice and Bob need a lot of randomness to perform the Bell test in the first place!. In QKD, we did not care since local randomness came for free: our challenge was to produce correlated and secret randomness!

It turns out that using Bell tests, there does however exist a way for Alice and Bob to produce <i>more</i> randomness that they started out with: this precisely is the concept of randomness expansion. In randomness expansion, we start with a small amount of perfect randomness and want to produce more it. It turns out that by performing Bell tests we can produce more - in fact, exponentially more - randomness than we started with!

While the analysis of such protocols is quite intricate, we will here explore the essential conceptual underpinning of randomness certification: if we violate a Bell inequality, then the outcomes are necessarily random!

<b>Important: JuliaBox has updated their kernel again - this notebook is tested for Julia 0.4.7! Before you run this notebook, click on "Kernel" in the menu above. Then go to "Change kernel" and select "Julia 0.4.7". Have fun!</b>

In [ ]:
include("source/main.jl");

<a id=Dist></a>
<h2 style="color:#00A6D6;">Randomness of the outputs</h2>

In this Julia sheet, we will explore the randomness of the outputs of a Bell inequality!

Remember that in the CHSH game we ask questions $x,y \in \{0,1\}$ to Alice and Bob, and collect outputs $a,b \in \{0,1\}$ as answers. Whatever strategy Alice and Bob employ there will be some probability $p(a,b|x,y)$ that Alice and Bob return answers $a$ and $b$ for questions $x$ and $y$. The winning probability for CHSH is then
$$
p_{\rm win} = \sum_{x,y} p(x,y) \sum_{a,b:x \cdot y =a\oplus b} p(a,b|x,y)
$$
Classically, the maximum is $3/4$ and quantumly $~0.853$.


<h3>No-signalling distributions</h3>
In quantum mechanics, it is not possible to communicate faster than light. This means that if the outputs $a$ and $b$ are produced in a timeframe that is too short to allow a signal to travel from Alice to Bob or vice versa, then then there is no way for Alice to send a signal to Bob by selecting inputs $x$ Bob observing a change in his output distribution (or vice versa). 

Formally, the no-signalling condition on $p(a,b|x,y)$ reads


$$
p(a|x) = \sum_{b} p(a,b|x,y) = \sum_{b} p(a,b|x,\hat{y})\qquad \forall a,x,y,\hat{y}
$$
and similarly

$$
p(b|y) = \sum_{a} p(a,b|x,y) = \sum_{a} p(a,b|\hat{x},y)\qquad \forall, a,x,\hat{x},y
$$

Let's suppose that Alice and Bob were indeed only limited by no-signalling. Can they achieve a higher winning probability in CHSH? 

Curiously, the answer turns out to be yes! Many "reasons" have been given why nature should not allow such superstrong CHSH winning probabilities, but stop at the quantum value of $~0.853$ (see e.g. Bell Nonlocality, Reviews of Modern Physics, 2014 for an overview).

<h3 style="color:#00A6D6;"> edX Exercise 1</h3>
Let's first check though that this is the case! We will ask Julia to optimize the winning probability of CHSH for us over all distributions that satisfy the no-signalling condition. 

What is the maximum no-signalling winning probability $p_{\rm win}$? What do you observe about the output distributions $p(a|x)$ and $p(b|y)$?

In [ ]:
using Convex;

# Define the variables p(a,b|x,y)
# We will use the convention that the entries of p correspond to p(00|00),p(01|00),p(10|00),p(11|00),p(00|01),...
p = Variable(16);

# Probabilities should be positive
constraints = [p >=0];

# And normalized: for all x and y we have p(00|x,y) + p(01|x,y) + p(10|x,y) + p(11|x,y) = 1
v = [1;1;1;1];
constraints += [v'*p[1:4] ==1 ];
constraints += [v'*p[5:8] == 1];
constraints += [v'*p[9:12] == 1];
constraints += [v'*p[13:16] == 1];

# The CHSH game as a vector c' * p gives the CHSH winning probability
c = [1;0;0;1;1;0;0;1;1;0;0;1;0;1;1;0]/4;

# No-signalling constraints from Bob to Alice p(a|x) - the others are implied by normalization
# p(0|0,0) - p(0|0,1) = 0
b00 = [1; 1; 0; 0; -1; -1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0];
constraints += [b00'*p == 0];

# p(0|1,0) - p(0|1,1) = 0
b01 = [0; 0; 0; 0; 0; 0; 0; 0; 1; 1; 0; 0; -1 ; -1; 0; 0];
constraints += [b01'*p == 0];

# No-signalling constraints form Alice to Bob p(b|y)
# p(0|0,0) - p(0|1,0) = 0
a00 = [1; 0; 1; 0; 0; 0; 0; 0; -1; 0; -1; 0; 0; 0; 0; 0];
constraints += [a00'*p == 0];

# p(0|0,1) - p(0|1,1) = 0
a01 = [0; 0; 0; 0; 1; 0; 1; 0; 0; 0; 0; 0; -1; 0; -1 ;0];
constraints += [a01'*p == 0];

# Define the objective function to be maximized subject to constraints
problem = maximize(c'*p,constraints);

# Run the linear programming solver
solve!(problem, SCSSolver(verbose=false));

# Print the solution
pwin = problem.optval;

print("The maximum no-signalling winning probability  is ", round(pwin,3), "\n")

# Print probability distribution
pVal = round(p.value,3);
print("The probabilties are:\n")
print("p(0,0|0,0) = ", pVal[1],"\n")
print("p(0,1|0,0) = ", pVal[2],"\n")
print("p(1,0|0,0) = ", pVal[3],"\n")
print("p(1,1|0,0) = ", pVal[4],"\n")
print("p(0,0|0,1) = ", pVal[5],"\n")
print("p(0,1|0,1) = ", pVal[6],"\n")
print("p(1,0|0,1) = ", pVal[7],"\n")
print("p(1,1|0,1) = ", pVal[8],"\n")
print("p(0,0|1,0) = ", pVal[9],"\n")
print("p(0,1|1,0) = ", pVal[10],"\n")
print("p(1,0|1,0) = ", pVal[11],"\n")
print("p(1,1|1,0) = ", pVal[12],"\n")
print("p(0,0|1,1) = ", pVal[13],"\n")
print("p(0,1|1,1) = ", pVal[14],"\n")
print("p(1,0|1,1) = ", pVal[15],"\n")
print("p(1,1|1,1) = ", pVal[16],"\n")

<h4>Randomness at the outputs</h4>

The key idea behind randomness certification is that if we observe a particular winning probability $p_{\rm win}$ then we can be sure that the outputs $a$ and $b$ are indeed random. As such, a particular value of the winning probability serves as a certificate that the output are random!

Your task will be to design a protocol to produce random numbers from an untrusted device! To make this easier, we will make the following simplifying assumptions
<ul>
<li> Even though Alice and Bob have no clue what's going on inside their devices, they know they behave exactly the same each time. That is, the devices have no memory, so we can test them ahead of time and then later produce random bits if we wish!</li>
<li> However, we are also more paranoid: we allow the adversary to prepare any no-signalling devices! Even those going beyond quantum mechanics. Note that quantum mechanics is no-signalling, so allowing the adversary to make no-signalling devices can only give him/her more power to cheat Alice and Bob!
</ul>


<h3 style="color:#00A6D6;"> edX Exercise 2</h3>
Let's first see what happens, if we know the outputs are NOT random!! We will impose the constraint that $p(a=0|x=0) = 1$. 

What is now the probability that Alice and Bob win the game? Take a look at the code that we wrote for the previous exercise. Can you give a simpler strategy for Alice and Bob than this no-signalling box that achieves the same winning probability?

In [ ]:
# Your Julia Code here!

You are highly encouraged to investigate the relationship between the randomness in the local outputs in relation to the winning probability.

<h3 style="color:#00A6D6;"> edX Exercise 3</h3>
What is the no-signalling winning probability (rounded to 3 digits) when we demand that
<ol>
<li>$p(a=0|x=0) = 3/4$?</li>
<li>$p(a=1|x=1) = 1/5$? </li>
<li>$p(b=0|y=1) = 6/7$? </li>
</ol>

Comparing the distributions $p(a|x)$ to $p(b|y)$ for each of the cases, what do you observe?

In [ ]:
# Your Julia Code here!


<h3 style="color:#00A6D6;"> edX Exercise 4</h3>
Now, let us add to the non-locality constraint a constraint on the winning probability. What is the maximum and minimum min-entropy $H_{\rm min}(A)$ of Alice's output $A$ compatible with the constraints? Compute these values for the following winning probabilities:

* $p_{\textrm{win}} = 1$
* $p_{\textrm{win}} = 0.85$
* $p_{\textrm{win}} = 0.75$
* $p_{\textrm{win}} = 0.5$

In [ ]:
# Your Julia Code here!


<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Week 3 Exercises for QuCryptoX</span> by David Elkouss and Stephanie Wehner for <a xmlns:cc="http://creativecommons.org/ns#" href="http://qutech.nl/edu/" property="cc:attributionName" rel="cc:attributionURL">QuTech Academy</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.